In [50]:
import datetime
import re

import pandas as pd
import numpy as np
import time 

from bs4 import BeautifulSoup
import requests

In [51]:
ricette = pd.read_excel('RICETTE.xlsx')
ricette.columns = ['RICETTA', 'LINK']

In [69]:
fact_tuples = []

source = 'GF'

for r in ricette.itertuples():
    
    time.sleep(3)
    rec = r.RICETTA.lower().capitalize()
    print(rec)
    

    # Get Page
    html = requests.get(r.LINK)
    soup = BeautifulSoup(html.text, 'html.parser')
    
    
    # --------------------------------------------------- #
    # Get Recipe Features 
    
    for j in soup.find_all("span", class_="gz-name-featured-data"):
    
        txt = j.text

        if txt.find('Difficoltà:') != -1:
            diff = txt.split(":")[-1].strip()
        
        if txt.find('Preparazione:') != -1:
            prep = txt.split(":")[-1].strip()
        
        if txt.find('Cottura:') != -1:
            cooktime = txt.split(":")[-1].strip()
        
        if txt.find('Dosi per:') != -1:
            dose = txt.split(":")[-1].strip()

        if txt.find('Costo:') != -1:
            cost = txt.split(":")[-1].strip()
            
    feat_tup = (diff, prep, cooktime, dose, cost, r.LINK, source)
    
    # --------------------------------------------------- #
    # Get Ingredients
    
    for i in soup.find_all("dd", class_="gz-ingredient"):
    
        fact = i.text.split()

        if fact[-1].strip() == 'q.b.':
            
            ingredient = " ".join(fact[:-1])
            unit_measure = fact[-1]
            quantity = None
            
            fact_tup = (rec, ingredient, unit_measure, quantity)
            full_tup = fact_tup + feat_tup
            fact_tuples.append(full_tup)
        
        elif re.search('^[0-9]+$', fact[-1].strip()):
            
            ingredient = " ".join(fact[:-1])
            unit_measure = None
            quantity = fact[-1]
            
            fact_tup = (rec, ingredient, unit_measure, quantity)
            full_tup = fact_tup + feat_tup
            fact_tuples.append(full_tup)
            
        else:
            
            ingredient = " ".join(fact[:-2])
            unit_measure = fact[-1]
            quantity = fact[-2]
            
            fact_tup = (rec, ingredient, unit_measure, quantity)
            full_tup = fact_tup + feat_tup
            fact_tuples.append(full_tup)
            
        break

Burger di patate
Carne alla pizzaiola
Club sandwich


KeyboardInterrupt: 

In [55]:
# Create Recipes Fact DF

cols = ['recipe_name', 'ingredient', 'unit_measure', 'quantity', 'preparation',
        'preparation_time', 'cooking_time', 'dose', 'cost_range', 'url', 'recipe_source']
df = pd.DataFrame(fact_tuples, columns = cols)

# Add timestamp

df['timestamp'] = datetime.datetime.now()

In [37]:
df = pd.DataFrame(fact_tuples, columns = cols)

In [ ]:
# Make dose numeric
df['dose'] = [int(x.split()[0]) for x in df['dose']]

In [68]:
df.to_excel('scraped_recipes.xlsx')

# Clean Up

In [64]:
# Make dose numeric
df['dose'] = [int(x.split()[0]) for x in df['dose']]

In [67]:
df['quantity'].unique()

array(['650', '140', '60', '50', '1', '40', '20', '2', None, '80', '100',
       '10', 'qualche', '300', '200', '6', '4', '15', '430', '500', '3',
       '450', '30', '130', '220', '800', '8', '400', '350', '5', '150',
       '600', '250', '70', '12', '900', '120', '320', '180', '4-5', '1,3',
       '½', '750', '110', '375', '7', '160', '90', '1,5', '170', '25',
       '18'], dtype=object)

In [70]:
# # Divide quantity by dose to make it for 1 person
# df['quantity'] = np.rint(df['quantity'].fillna(0).astype(int) / df['dose'])